In [1]:
import simpy
import random

# Define constants based on the provided parameters
NUM_CLIENTS = random.randint(100, 200)
NUM_DATACENTERS = 1
NUM_HOSTS = 2

# Cloud node parameters
CLOUD_NODES = 10
CLOUD_CPU_POWER = (3000, 5000)  # MIPS
CLOUD_RAM = 8  # GB
CLOUD_STORAGE = 1  # TB
CLOUD_BANDWIDTH = 0.5  # Gb/s

# Fog node parameters
FOG_NODES = 10
FOG_CPU_POWER = (1000, 2800)  # MIPS
FOG_RAM = 2  # GB
FOG_STORAGE = 0.5  # TB
FOG_BANDWIDTH = 1  # Gb/s

# Task parameters
TASK_LENGTH = (2000, 56000)  # MI
DATA_SIZE = (400, 600)  # MB
TASK_SETS = [300, 600, 900, 1200, 1500]  # Different sets of task numbers

class Node:
    def __init__(self, env, cpu_power, ram, storage, bandwidth):
        self.env = env
        self.cpu_power = cpu_power
        self.ram = ram
        self.storage = storage
        self.bandwidth = bandwidth
        self.queue = simpy.Resource(env, capacity=1)

    def process_task(self, task_length, data_size):
        # Processing time is based on task length and CPU power
        processing_time = task_length / self.cpu_power
        yield self.env.timeout(processing_time)

def generate_tasks(env, nodes, task_length_range, data_size_range, num_tasks):
    task_start_times = []
    task_end_times = []
    
    for _ in range(num_tasks):
        task_length = random.randint(*task_length_range)
        data_size = random.randint(*data_size_range)
        node = random.choice(nodes)
        task_start_times.append(env.now)
        env.process(handle_task(env, node, task_length, data_size, task_end_times))
        yield env.timeout(1)  # Small delay to simulate task arrival interval
    
    yield env.timeout(0)  # Yield control to allow all tasks to be processed
    yield env.all_of([env.timeout(1) for _ in range(num_tasks)])  # Wait for all tasks to be completed
    makespan = max(task_end_times) - min(task_start_times)
    return makespan

def handle_task(env, node, task_length, data_size, task_end_times):
    with node.queue.request() as request:
        yield request
        yield env.process(node.process_task(task_length, data_size))
        task_end_times.append(env.now)

def main():
    env = simpy.Environment()

    # Create cloud nodes
    cloud_nodes = [
        Node(env, random.randint(*CLOUD_CPU_POWER), CLOUD_RAM, CLOUD_STORAGE, CLOUD_BANDWIDTH)
        for _ in range(CLOUD_NODES)
    ]

    # Create fog nodes
    fog_nodes = [
        Node(env, random.randint(*FOG_CPU_POWER), FOG_RAM, FOG_STORAGE, FOG_BANDWIDTH)
        for _ in range(FOG_NODES)
    ]

    all_nodes = cloud_nodes + fog_nodes

    # Track makespan for different sets of tasks
    makespans = {}

    for num_tasks in TASK_SETS:
        env = simpy.Environment()
        cloud_nodes = [
            Node(env, random.randint(*CLOUD_CPU_POWER), CLOUD_RAM, CLOUD_STORAGE, CLOUD_BANDWIDTH)
            for _ in range(CLOUD_NODES)
        ]
        fog_nodes = [
            Node(env, random.randint(*FOG_CPU_POWER), FOG_RAM, FOG_STORAGE, FOG_BANDWIDTH)
            for _ in range(FOG_NODES)
        ]
        all_nodes = cloud_nodes + fog_nodes
        
        makespan = env.process(generate_tasks(env, all_nodes, TASK_LENGTH, DATA_SIZE, num_tasks))
        env.run()
        makespans[num_tasks] = makespan.value

    # Output the makespan for each set of tasks
    for num_tasks, makespan in makespans.items():
        print(f"Makespan for {num_tasks} tasks: {makespan}")

if __name__ == "__main__":
    main()


Makespan for 300 tasks: 300.831254412803
Makespan for 600 tasks: 600.7392801251956
Makespan for 900 tasks: 899.6685962373372
Makespan for 1200 tasks: 1200.961428199114
Makespan for 1500 tasks: 1499.4296144193474
